# Making CDR Ready To Work With A Real Quantum Computer


This post appeared first on [Medium](https://towardsdatascience.com/making-cdr-ready-to-work-with-a-real-quantum-computer-fcdf270a6730) and my weekly [email-course](https://pyqml.substack.com/p/making-cdr-ready-to-work-with-a-real?s=w).


Clifford Data Regression (CDR) is a promising quantum error mitigation method. In the previous post, we effectively adapted the code from the open-source library Mitiq. As a result, we only had to apply minor adjustments to connect to the IBM Quantum cloud and to reduce the noise of a real quantum computer.

However, it takes a whole day to complete the method. It doesn't take so long because it takes so much computing time. But it takes so long because the quantum circuits we submitted to the IBM cloud have been waiting a very long time to be executed.

IBM offers free access to (some of) their quantum devices. But, of course, these are shared devices. When you send a quantum circuit to a quantum system on the IBM cloud, it enters a queue with jobs from other users before eventual execution. Depending on the number of enqueued jobs, it may take hours until a job runs.

During our execution of the CDR, we submitted eleven circuits to the IBM cloud. The problem is that we didn't send them all immediately. But we always waited until one job was done before we sent the next. As a result, our waiting time added up significantly.

Fortunately, IBM allows us to send multiple circuits concurrently. This should reduce our overall waiting time significantly.

In the last post's code, we use the `execute_with_cdr` method we import from mitiq. We call it with a custom executor as a parameter--the `real_jakarta` executor. This executor is a function that takes the quantum circuit, runs it, and returns the expectation value of the observable of interest.

Let's have a brief look at the code of the `execute_with_cdr` function that you find [here](https://github.com/unitaryfund/mitiq/blob/master/mitiq/cdr/cdr.py). At line 155, we call the executor's `evaluate` function and pass as the parameters a list of circuits and the observable.

The list of circuits is the training circuits plus the original circuit whose expectation value we want to calculate. The CDR method builds on learning from data how to mitigate the noise. We see that we can specify the number of training circuits we want to use on closer inspection. The default value is `10`. So, we call the evaluate function with a list of eleven circuits.

The executor (see its code [here](https://github.com/unitaryfund/mitiq/blob/master/mitiq/executor/executor.py)) loops through the list and "executes" each circuit by calling the function we provided on it.

Let's briefly revisit that provided function to see how we execute a circuit.

```python
# ...

mapped_circuit = transpile(circ, backend=jakarta)
qobj = assemble(mapped_circuit, backend=jakarta, shots=shots)

# execute the circuit
job = jakarta.run(qobj)

counts = job.result().get_counts()

# ...
```

First, we transpile, assemble, and run the circuit on the real quantum device (Jakarta). Then, we directly obtain the's result inside the executor and, thus, inside the loop. That's the problem because `result()` is a blocking method. It does not return anything, but it waits until the job has finished. So, it waits as long as our circuit waits in the queue. Consequently, the whole loop doesn't proceed. But it waits for the result before submitting the next circuit for execution.

To prevent our code from waiting, we must submit all circuits to the IBM cloud before using their results. Therefore, we need to take apart the `execute_with_cdr` function.

In the first step, we prepare the training circuits. We separate this from the rest of the code because we need these circuits before and after execution.

In [1]:
from qiskit import QuantumCircuit
from mitiq.interface.mitiq_qiskit import qiskit_utils
from mitiq import Observable, PauliString
from typing import Any, Callable, Optional, Sequence, Union
import numpy as np
from scipy.optimize import curve_fit

from mitiq import Executor, Observable, QPROGRAM, QuantumResult
from mitiq.cdr import (
    generate_training_circuits,
    linear_fit_function,
    linear_fit_function_no_intercept,
    is_clifford,
)
from mitiq.zne.scaling import fold_gates_at_random


def prepare_training_circuits(
    circuit: QPROGRAM,
    observable: Optional[Observable] = None,
    *,
    simulator: Union[Executor, Callable[[QPROGRAM], QuantumResult]],
    num_training_circuits: int = 10,
    fraction_non_clifford: float = 0.1,
    fit_function: Callable[..., float] = linear_fit_function,
    num_fit_parameters: Optional[int] = None,
    scale_factors: Sequence[float] = (1,),
    **kwargs: Any,
):
    """
    Returns a list of the training circuits
    """
    
    method_select = kwargs.get("method_select", "uniform")
    method_replace = kwargs.get("method_replace", "closest")
    random_state = kwargs.get("random_state", None)
    kwargs_for_training_set_generation = {
        "sigma_select": kwargs.get("sigma_select"),
        "sigma_replace": kwargs.get("sigma_replace"),
    }

    if num_fit_parameters is None:
        if fit_function is linear_fit_function:
            num_fit_parameters = 1 + len(scale_factors)
        elif fit_function is linear_fit_function_no_intercept:
            num_fit_parameters = len(scale_factors)
        else:
            raise ValueError(
                "Must provide `num_fit_parameters` for custom fit function."
            )


    if not isinstance(simulator, Executor):
        simulator = Executor(simulator)

    # Check if circuit is already Clifford
    if is_clifford(circuit):
        return simulator.evaluate(circuit, observable)[0].real

    # Generate training circuits.
    training_circuits = generate_training_circuits(
        circuit,
        num_training_circuits,
        fraction_non_clifford,
        method_select,
        method_replace,
        random_state,
        kwargs=kwargs_for_training_set_generation,
    )

    return training_circuits

/usr/local/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


We stripped this function to contain only the parts required for generating the training circuits. But those parts we took over remain unmodified (see the [original function](https://github.com/unitaryfund/mitiq/blob/master/mitiq/cdr/cdr.py)).

So, let's now use this function to create the list of circuits. For that purpose, we first reuse a few things from the previous posts, such as the `get_circuit()` function that creates the quantum circuit whose observable expectation value we aim to calculate. Further, we define this observable (`obs`), and the noiseless simulator (`sim`).

Once we prepared the circuits, we reorganized them similar to the original Mitiq code. 

In [2]:
def get_circuit():
    qc = QuantumCircuit(2)

    # CDR works better if the circuit is not too short. So we increase its depth.
    for i in range(5): 
        qc.h(0) # Clifford
        qc.h(1) # Clifford
        qc.rz(1.75, 0)
        qc.rz(2.31, 1)
        qc.cx(0,1) # Clifford
        qc.rz(-1.17, 1)
        qc.rz(3.23, 0)
        qc.rx(pi/2, 0) # Clifford
        qc.rx(pi/2, 1) # Clifford
        
    return qc


def sim(qc):
    return qiskit_utils.execute_with_shots(qc, obs.matrix(), 4096)

obs = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))

scale_factors: Sequence[float] = (1,)
scale_noise: Callable[[QPROGRAM, float], QPROGRAM] = fold_gates_at_random

training_circuits = prepare_training_circuits(
    get_circuit(),
    observable=obs.matrix(),
    simulator=sim,
    scale_factors=scale_factors,
)


# [Optionally] Scale noise in circuits.
all_circuits = [
    [scale_noise(c, s) for s in scale_factors]
    for c in [get_circuit()] + training_circuits  # type: ignore
]


to_run = [circuit for circuits in all_circuits for circuit in circuits]
all_circuits_shape = (len(all_circuits), len(all_circuits[0]))

We store the circuits we want to run in the `to_run` variable. So, let's now create a function that submits these circuits to the IBM cloud all at once (without waiting for a result). 

In [3]:
from qiskit import QuantumCircuit, transpile, assemble

def submit_to_jakarta(
    to_run,
    obs: np.ndarray,
    shots: int,
    backend
):
    
    def submit_executor(circuit):

        # Avoid mutating circuit
        circ = circuit.copy()
        # we need to modify the circuit to measure obs in its eigenbasis
        # we do this by appending a unitary operation
        # obtains a U s.t. obs = U diag(eigvals) U^dag
        eigvals, U = np.linalg.eigh(obs)
        circ.unitary(np.linalg.inv(U), qubits=range(circ.num_qubits))

        circ.measure_all()

        mapped_circuit = transpile(circ, backend=backend)
        qobj = assemble(mapped_circuit, backend=backend, shots=shots)

        # execute the circuit
        job = jakarta.run(qobj)
        return job

    
    # cast executor and simulator inputs to Executor type
    executor = Executor(submit_executor)
    jobs = executor.evaluate(to_run, obs)
    return jobs

The `submit_to_jakarta` function takes the `to_run` list, the observable, the number of runs per circuit (`shots`), and the `backend` we want to run the circuits. Inside this function, we declare another function `submit_executor`. This is the callback function we use as the executor. In the implementation of this function, we only prepare the circuit (transpile and assemble) and send it to the `backend` backend. The executor does not return the result but a `job`. This is a local reference to a quantum circuit submitted to a given backend. The `submit_to_jakarta` function collects all the `jobs` and returns them.

Before we can use this function, we need to connect to our account and get a provider (you need to use different parameter values to connect to `ibmq_quito`) that allows us to get access to a backend, here `jakarta`. [This post](https://towardsdatascience.com/how-to-run-code-on-a-real-quantum-computer-c1fc61ff5b4) explains how to do this in more detail.


In [4]:
# load IBMQ Account data
from qiskit import IBMQ

# replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
provider = IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')

Let's see what happens when we call this function.

In [13]:
jobs = submit_to_jakarta(
    to_run,
    obs.matrix(),
    4096,
    jakarta
)
jobs

/tmp/ipykernel_86/179072001.py:26: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  job = jakarta.run(qobj)


The output is a list of job objects. Head over to your [IBM Quantum console](https://quantum-computing.ibm.com/). When you click "view all" in "recent jobs," you'll see that these jobs have arrived at IBM and they are pending. You also see the approximated time of execution. In my case, they're not going to run today at all because there are more than 140 other circuits in the queue waiting.

![IBMQ](./assets/ibmq2.png)

If I needed any more motivation for what we do today, this would be it. 

At least, waiting for these jobs to run gives me  enough time to complete this post.



Of course, you don't want to keep your computer up and running while we wait for the results. So, we need to persist these jobs. Fortunately, each job has an id we can use to get it back later. If you use a Jupyter notebook, you can print the ids into the output of a cell and save them alongside your notebook.


In [16]:
for job in jobs:
    print("'{}',".format(job.job_id()))

'6218d0f147f37310a6377ee0',
'6218d0f3a2eeaa7a80ad27c8',
'6218d0f48c4ac898e4687d1c',
'6218d0f5a2eeaa782fad27c9',
'6218d0f7f0b807c602db65b0',
'6218d0f8a16487a0ade6eb39',
'6218d0fa3895be83e73e214f',
'6218d0fbf0b8079824db65b1',
'6218d0fc5ac21f18a4473086',
'6218d0fe976d9fc87220be83',
'6218d0ff3f99d49bef399494',


We see a list of eleven job ids.

You can use them to obtain a new job object through the `jakarta.retrieve_job("JOB_ID")` function. So, let's put these into an array that we can work with. Here, I use the ids of previous runs.

In [5]:
finished_job_ids = [
    "6216307c89a5f0ba67accdc8",
    "621633e989a5f0085faccdca",
    "62163503bc4128148fb6acca",
    "6216358612d1b60020b8202c",
    "6216358612d1b60020b8202c",
    "6216362557bc5470c462c379",
    "6216373e12d1b6d592b8202f",
    "62163784538b7855738ee875",
    "62163824ffea1318711becd8",
    "621638d8e206decd5890bbb1",
    "6216398261cbfc3bfc62f373"
]

We can easily map the list of ids to a list of the actual jobs.

In [6]:
finished_jobs = list(map(jakarta.retrieve_job, finished_job_ids))
finished_jobs

Once these jobs are complete, we can obtain their results without further waiting.

In [8]:
finished_jobs[0].result()

Result(backend_name='ibmq_jakarta', backend_version='1.0.28', qobj_id='471fdd0a-c273-4ea4-aded-28f0ed4b0ed7', job_id='6216307c89a5f0ba67accdc8', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 238, '0x1': 41, '0x2': 49, '0x3': 696}), header=QobjExperimentHeader(qubit_labels=[['q', 0], ['q', 1], ['q', 2], ['q', 3], ['q', 4], ['q', 5], ['q', 6]], n_qubits=7, qreg_sizes=[['q', 7]], clbit_labels=[['meas', 0], ['meas', 1]], memory_slots=2, creg_sizes=[['meas', 2]], name='circuit-3333', global_phase=4.71238898038469, metadata={}))], date=2022-02-23 13:17:22+00:00, status=Successful completion, status=QobjHeader(backend_name='ibmq_jakarta', backend_version='1.0.28'), execution_id='ec06e180-94aa-11ec-ac54-bc97e15b08d0', time_taken=5.931757688522339, error=None, client_version={'qiskit': '0.33.0'})

So, let's write another function that processes the job results. It takes the jobs and the observable. Again, we define an executor inside this function that does the actual work.

In [7]:
def process_jobs(
    jobs,
    obs: np.ndarray
):
    
    def process_executor(job):
        eigvals, U = np.linalg.eigh(obs.matrix())
        counts = job.result().get_counts()
        expectation = 0

        for bitstring, count in counts.items():
            expectation += (
                eigvals[int(bitstring[0 : get_circuit().num_qubits], 2)] * count / job.result().results[0].shots
            )
        return expectation


    results = list(map(process_executor, jobs))
    return results

exp_values = process_jobs(finished_jobs, obs)
exp_values

[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]
[-2.01556444 -2.01556444  2.01556444  2.01556444]


[0.9172392848406065,
 0.5629408486360803,
 0.4802712135216911,
 0.8463795975997012,
 0.8463795975997012,
 0.7204068202825363,
 0.6259272372946627,
 0.7322167681560205,
 0.7400900667383433,
 0.7440267160295047,
 0.1417193744818105]

When we process the jobs, we receive a list of expectation values. Similar to Mitiq's original function, we need to combine these with the noise-free measurements we get from the simulator. So, let's do these steps, too, to get the error-mitigated measurement.

In [11]:
simulator = Executor(sim)
num_fit_parameters = 1 + len((1,),)
results = simulator.evaluate(training_circuits, obs.matrix())
ideal_results = np.array(results)

noisy_results = np.array(exp_values).reshape(all_circuits_shape)

# Do the regression.
fitted_params, _ = curve_fit(
    lambda x, *params: linear_fit_function(x, params),
    noisy_results[1:, :].T,
    ideal_results,
    p0=np.zeros(num_fit_parameters),
)
print (noisy_results[1:, :].T, noisy_results[0, :], fitted_params)

jakarta_mitigated_measurement = linear_fit_function(noisy_results[0, :], fitted_params)
jakarta_mitigated_measurement

[[0.56294085 0.48027121 0.8463796  0.8463796  0.72040682 0.62592724
  0.73221677 0.74009007 0.74402672 0.14171937]] [0.91723928] [-0.13531499  1.12711203]


1.0029958033732036

So, let's calculate the ideal (noise-free) and the unmitigated measurement. When we created the list of circuits to run, the very first was the unmodified circuit. We can use this to calculate the unmitigated measurement, too.

Finally, we can calculate and print the key performance indicators of the mitigation.

In [29]:
ideal_measurement = obs.expectation(get_circuit(), sim).real
jakarta_unmitigated_measurement = obs.expectation(get_circuit(), lambda x: exp_values[0]).real
jakarta_error_unmitigated = abs(jakarta_unmitigated_measurement-ideal_measurement)
jakarta_error_mitigated = abs(jakarta_mitigated_measurement-ideal_measurement)

print("Error (unmitigated):", jakarta_error_unmitigated)
print("Error (mitigated with CDR):", jakarta_error_mitigated)
print("Relative error (unmitigated):", (jakarta_error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):", jakarta_error_mitigated/ideal_measurement)
print(f"Error reduction with CDR: {(jakarta_error_unmitigated-jakarta_error_mitigated)/jakarta_error_unmitigated :.1%}.")

Error (unmitigated): 0.06987552491811488
Error (mitigated with CDR): 0.026457842276776833
Relative error (unmitigated): 0.07078763708873381
Relative error (mitigated with CDR): 0.026803206694106713
Error reduction with CDR: 62.1%.


The results show an error reduction of more than 60%. Even though this is not as good as the mitigation of the simulated measurements we achieved in the previous posts, it is a decent improvement over the unmitigated result.